In [1]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns

In [2]:
# api_key = "Enter api key"
api_key = ""
# Political
# channel_id = ["channel id"]


youtube = build("youtube", "v3", developerKey=api_key)

In [3]:
#func to get channel stats
def get_chann_stats(youtube, channel_id):
    all_data=[]
    request = youtube.channels().list(
              part='snippet, contentDetails, statistics',
              id=','.join(channel_id))
    response = request.execute()
    for i in range(len(response['items'])):
        data=dict(Channel_name=response['items'][i]['snippet']['title'],
                 Subscribers=response['items'][i]['statistics']['subscriberCount'],
                 Views=response['items'][i]['statistics']['viewCount'],
                 Total_videos=response['items'][i]['statistics']['videoCount'],
                 Playlist_id=response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    return all_data

In [4]:
channel_Stats= get_chann_stats(youtube, channel_id)

In [5]:
channel_data=pd.DataFrame(channel_Stats)

In [6]:
channel_data=channel_data.sort_values(by=['Channel_name'])

In [7]:
len(channel_data)

19

In [8]:
channel_data

,Channel_name,Subscribers,Views,Total_videos,Playlist_id
1,24 News HD,6980000,3001410325,146955,UUcmpeVbSSQlZRvHfdC-CRwg
3,92 News HD,4670000,1526861541,73889,UUsgC5cbz3DE2Shh34gNKiog
14,ARY News,12000000,6459390407,74915,UUMmpLL2ucRHAXbNHiCPyIyg
4,Abbtakk,901000,213101405,87378,UU5mwDEzm4FzXKoHPBDnuUQQ
18,Capital TV,1820000,926225626,60745,UU-an6pHQfTVZXgrLP0p1EGA
17,City 41,948000,203266980,103197,UU6UW8OOQQBGdwmnaYLh-qdw
16,City 42,2180000,644279132,149407,UUdTup4kK7Ze08KYp7ReiuMw
6,Dunya News,6260000,3030622951,114965,UUnMBV5Iw4WqKILKue1nP6Hg
7,Express News,3300000,879735094,61594,UUTur7oM6mLL0rM2k0znuZpQ
15,GNN,2450000,927852028,61824,UU35KuZBNIj4S5Ls0yjY-UHQ


In [9]:
channel_data.to_csv("Pakistani News Channels Data.csv")

In [ ]:
# channel_data.dtypes

In [ ]:
# channel_data['Subscribers']=pd.to_numeric(channel_data['Subscribers'])
# channel_data['Views']=pd.to_numeric(channel_data['Views'])
# channel_data['Total_videos']=pd.to_numeric(channel_data['Total_videos'])
# channel_data.dtypes


In [ ]:
# sns.set(rc={'figure.figsize':(20,20)})
# ax= sns.barplot(x='Channel_name', y='Subscribers', data=channel_data)

In [ ]:
# sns.set(rc={'figure.figsize':(25,25)})
# ax= sns.barplot(x='Channel_name', y='Views', data=channel_data)

In [ ]:
# sns.set(rc={'figure.figsize':(5,5)})
# ax= sns.barplot(x='Channel_name', y='Total_videos', data=channel_data)

In [ ]:
# channel_data

In [10]:
playlist_id = channel_data.loc[channel_data['Channel_name']=="24 News HD",'Playlist_id'].iloc[0]

In [11]:
playlist_id

'UUcmpeVbSSQlZRvHfdC-CRwg'

In [12]:
#Function to get Video IDs
def get_video_ids(youtube, playlist_id):
    request = youtube.playlistItems().list(
            part='contentDetails',
            playlistId =playlist_id,
            maxResults = 50)
    response = request.execute()
    
    video_ids=[]
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
    
    next_page_token = response['nextPageToken']
    more_pages= True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                    part='contentDetails',
                    playlistId =playlist_id,
                    maxResults = 50,
                    pageToken= next_page_token)
            response = request.execute()
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            next_page_token = response.get('nextPageToken')
    
#     return len(video_ids)
    return video_ids

In [13]:
video_ids= get_video_ids(youtube, playlist_id)

In [14]:
len(video_ids)

19961

In [17]:
# video details
def get_video_details(youtube, video_ids):
    
    all_video_stats=[]
    for i in range(0,len(video_ids),50):
        request = youtube.videos().list(
                    part='snippet, statistics',
                    id=','.join(video_ids[i:i+50]))
        response = request.execute() 
        
        for video in response['items']:
            video_stats=dict(Title=video["snippet"]["title"],
                            Published_Date=video["snippet"]["publishedAt"],
                            Description= video["snippet"]["description"],
#                             Tags= video["snippet"]["tags"],
                            Views = video["statistics"]["viewCount"],
#                              Favorites = video["statistics"]["favoriteCount"],
#                             Likes = video["statistics"]["likeCount"])
                            Likes = video["statistics"]["likeCount"])
#                            Comments = video["statistics"]["commentCount"])
            all_video_stats.append(video_stats)
        
    return all_video_stats



In [18]:
get_video_details(youtube, video_ids)

[{'Title': 'Election Will Be On 14th May | 7am News Headlines | 5 May 2023 | 24 News HD',
  'Published_Date': '2023-05-05T02:15:02Z',
  'Description': 'Election Will Be On 14th May | 7am News Headlines | 5 May 2023 | 24 News HD\n\n24 News HD Is One Of The Leading News Channels Of Pakistan Bringing You The Latest Current Affairs From Pakistan And Around The World.  Live National Assembly ,Senate Session And Current Affairs \n\nPM Shehbaz Sharif | Former Prime Minister Imran Khan | Nasim Zehra @ 8 | Sajjad Jani | 10 Tak | 24 Special | Inkashaf | Nazir Ghazi Pakistan Latest and Breaking News | Live News | Former Prime Minister Imran Khan | News Headlines | National Assembly of Pakistan | Senate | President Arif Alvi | Q K Jamhuriat Hai | Najam Sethi Show | Exclusive News | News Bulletin\n\nFollow us on:-\n►Website: https://www.24newshd.tv/\n►Twitter: https://twitter.com/24NewsHD \u200b\n►Facebook: http://www.facebook.com//24NewsHD.tv\n►Instagram: https://www.instagram.com/24newshd.pk\n►Yo

In [20]:
video_details=get_video_details(youtube, video_ids)

In [21]:
video_data=pd.DataFrame(video_details)

In [22]:
video_data

,Title,Published_Date,Description,Views,Likes
0,Election Will Be On 14th May | 7am News Headli...,2023-05-05T02:15:02Z,Election Will Be On 14th May | 7am News Headli...,536,8
1,LIVE | Noor-e-Sahar With Justice (R) Nazeer Ah...,2023-05-05T02:03:35Z,LIVE | Noor-e-Sahar With Justice (R) Nazeer Ah...,853,27
2,Heavy Rain Prediction | 6am News Headlines | 5...,2023-05-05T01:15:02Z,Heavy Rain Prediction | 6am News Headlines | 5...,849,19
3,PTI Workers In Trouble | 5am News Headlines | ...,2023-05-05T00:15:04Z,PTI Workers In Trouble | 5am News Headlines | ...,997,18
4,PTI In Big Trouble | 4am News Headlines | 5 Ma...,2023-05-04T23:15:02Z,PTI In Big Trouble | 4am News Headlines | 5 Ma...,1308,16
...,...,...,...,...,...
19956,"Huge Order, Good News For Hamza Shahbaz",2022-04-23T09:00:14Z,"Huge Order, Good News For Hamza Shahbaz \n24 N...",84691,613
19957,"Advance Salary Before Eid, Good News For Govt ...",2022-04-23T08:45:12Z,"Advance Salary Before Eid, Good News For Govt ...",33817,452
19958,Breaking News! Governor Punjab Umar Sarfraz Ch...,2022-04-23T08:30:03Z,Breaking News! Governor Punjab Umar Sarfraz Ch...,5628,30
19959,Big News Shortly | 1pm News Headlines | 23 Apr...,2022-04-23T08:10:23Z,Big News Shortly | 1pm News Headlines | 23 Apr...,38576,349


In [23]:
video_data.to_csv("24 News HD.csv")

In [164]:
video_data['Published Date']=pd.to_datetime(video_data['Published_Date']).dt.date

In [165]:
video_data

,Title,Published_Date,Description,Views,Likes,Comments,Published Date
0,Kamry ki tiles or lights lag gai😍|| hussain ny...,2023-03-17T13:04:35Z,I hope you enjoyed the video❤️\n\nPlease do LI...,362743,23797,340,2023-03-17
1,Itny din baad mini house pohonchy❤️or yeh ho g...,2023-03-16T17:40:37Z,I hope you enjoyed the video❤️\n\nPlease do LI...,241076,19939,407,2023-03-16
2,Hamain hotel mai rehna parha🤩|| kamra kab tak ...,2023-03-15T12:28:15Z,I hope you enjoyed the video❤️\n\nPlease do LI...,294741,22608,331,2023-03-15
3,Aj mai ny apna poora kamra toor diya😳😱🔨,2023-03-14T13:47:38Z,I hope you enjoyed the video❤️\n\nPlease do LI...,293313,24529,405,2023-03-14
4,Say yes to yusra for 24 hour😍|| Mera keemti pe...,2023-03-06T14:32:17Z,I hope you enjoyed the video❤️\n\nPlease do LI...,438403,48039,1356,2023-03-06
...,...,...,...,...,...,...,...
69,Day out with my wife❤️,2022-02-16T09:06:21Z,Hey guys…👋🏻\nSab ko comment mai batana hy vlog...,1695170,126817,2676,2022-02-16
70,Eating burger in 5 minutes challenge | Hilario...,2022-02-10T09:43:49Z,Welcome back to the another video.Hope so you ...,1263431,111155,2375,2022-02-10
71,Bhai ka walima❤️ | vlog 1,2022-02-07T14:49:07Z,,1806274,110458,1963,2022-02-07
72,Our first challenge video ( flip the bottle ) ...,2022-01-28T17:05:46Z,Welcome back to the another video.Hope so you ...,1054875,105888,3057,2022-01-28


In [27]:
video_data['Month']=pd.to_datetime(video_data['Published Date']).dt.strftime('%B')
video_data['Day']=pd.to_datetime(video_data['Published Date']).dt.strftime('%A')
video_data['Week Day']=pd.to_datetime(video_data['Published Date']).dt.strftime('%w')
video_data['Date']=pd.to_datetime(video_data['Published Date']).dt.strftime('%d')
video_data['Year']=pd.to_datetime(video_data['Published Date']).dt.strftime('%Y')

In [28]:
video_data

,Title,Published_Date,Description,Views,Published Date,Month,Day,Week Day,Date,Year
0,Pyar Zindagi Aur Karachi ft Sania Saeed | Epis...,2023-02-23T16:15:02Z,This is the official YouTube channel for Fuchs...,14144,2023-02-23,February,Thursday,4,23,2023
1,Top 5 Dramas of the week | Tere Bin | Mujhe Py...,2023-02-22T14:00:15Z,This is the official YouTube channel for Fuchs...,5798,2023-02-22,February,Wednesday,3,22,2023
2,Kuch Ankahi | Tere Bin | Yunhi | Sar e Rah | M...,2023-02-21T14:00:33Z,0:00 - Intro\n3:57 - Meri Shehzadi \n10:28 - W...,7716,2023-02-21,February,Tuesday,2,21,2023
3,Lame Questions Guaranteed with the cast of MBG...,2023-02-20T15:30:02Z,This is the official YouTube channel for Fuchs...,19224,2023-02-20,February,Monday,1,20,2023
4,Drama Gup | Tere Bin | Mujhe Pyaar Hua Tha | H...,2023-02-17T18:40:57Z,Drama Gup is a review on the latest episode of...,15075,2023-02-17,February,Friday,5,17,2023
...,...,...,...,...,...,...,...,...,...,...
1338,blush | FUCHSIA,2015-05-03T15:41:00Z,Share some love and subscribe now: http://bit....,159,2015-05-03,May,Sunday,0,03,2015
1339,contouring Sam | FUCHSIA,2015-05-02T15:28:14Z,Share some love and subscribe now: http://bit....,134,2015-05-02,May,Saturday,6,02,2015
1340,DIY 1 | FUCHSIA,2015-03-30T17:16:21Z,Share some love and subscribe now: http://bit....,110,2015-03-30,March,Monday,1,30,2015
1341,Green casserole | FUCHSIA,2015-03-29T15:03:44Z,Share some love and subscribe now: http://bit....,201,2015-03-29,March,Sunday,0,29,2015


In [ ]:
videos_per_month = video_data.groupby('Month', as_index=False).size()

In [ ]:
videos_per_month

In [ ]:
sort_order=['Jan', 'Feb', 'March', 'Apr', 'May', 'June', 'July',
            'August','September', 'October','November','December']

In [ ]:
videos_per_month.index=pd.CategoricalIndex(videos_per_month['Month'], categories=sort_order, ordered =True)

In [ ]:
videos_per_month=videos_per_month.sort_index()

In [ ]:
videos_per_month

In [ ]:
channel_data.to_csv("Channel Data.csv")